## Importing Libraries

In [1]:
import os
import re
from stop_words import get_stop_words
import numpy as np
from sklearn import model_selection
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from pylab import rcParams
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

## Importing all the documents

In [2]:
X = [] # X is represented as (document's name,text of document)
Y = [] # Y is represented as the newsgroup category of the corresponding X data

for category in os.listdir('20_newsgroups'):
    print(category)
    for document in os.listdir('20_newsgroups/'+category):
        with open('20_newsgroups/'+category+'/'+document, "r") as f:
            X.append((document,f.read()))
            Y.append(category)

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


## Spliting the Data in Training and Testing Data

In [3]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X,Y)

## Function to get the Features Set on the basis of Training Data

In [4]:
def get_feature_set(x_train):
    
    #Getting list of stop words
    stop_words = get_stop_words('english')
    d = {}
    
    #Iterating through all the documents in the trainng set 
    for i in range(len(x_train)):
        #Spliting the documents into words list
        words = re.split(r'\W+', X[i][1])
        #Changeing all the words to lower case to maintain uniformity
        words = [word.lower() for word in words]
        for word in words:
            #Checking if the words is a stop words or a number
            if (word in stop_words) or (not word.isalpha()):
                continue
            d[word] = d.get(word, 0) + 1

    keys = np.array(list(d.keys()))
    values = np.array(list(d.values()))
    
    # Getting the top 1000 words with the highest frequency
    ind = values.argsort()[::-1]
    ind = ind[:2000]
    values = values[ind]
    keys = keys[ind]

    return keys

## Function to split  a Document into Words List

In [5]:
def get_word_list(text):
    words = re.split(r'\W+', text)
    words = [word.lower() for word in words]
    return words

## Function to get Data in the form of 2D Array

In [6]:
def get_2D_data(x, feature_set):
    
    data = []
    n = len(feature_set)
    
    #Iterating through all the documents in x
    for i in range(len(x)):
        #Spliting the documents into words list
        words = get_word_list(x[i][1])
        l = np.zeros(n, dtype = int)
        for word in words:
            #Checking if the word is present in the feature set or not
            ind = np.where(feature_set == word)
            l[ind] = l[ind] + 1
        data.append(l)
    return data              

## 1. Perform Test Classification using Multinomial Naive Bayes (already implemented in sklearn)

### Training

In [7]:
feature_set = get_feature_set(x_train)

In [8]:
x_train_2d = get_2D_data(x_train, feature_set)

In [9]:
clf = MultinomialNB()
clf.fit(x_train_2d, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Testing

In [10]:
x_test_2d = get_2D_data(x_test, feature_set)

In [11]:
y_pred = clf.predict(x_test_2d)
print("---------------------------------------------------------------------------")
print("Classification Report")
print(classification_report(y_test,y_pred))
print("---------------------------------------------------------------------------")
print("Confusion Matrix")
print(confusion_matrix(y_test,y_pred))
print("---------------------------------------------------------------------------")
print("Mean Accuracy: ", clf.score(x_test_2d, y_test, sample_weight=None))
print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
Classification Report
                          precision    recall  f1-score   support

             alt.atheism       0.82      0.74      0.77       253
           comp.graphics       0.77      0.72      0.74       249
 comp.os.ms-windows.misc       0.83      0.06      0.12       233
comp.sys.ibm.pc.hardware       0.57      0.87      0.69       263
   comp.sys.mac.hardware       0.77      0.90      0.83       252
          comp.windows.x       0.81      0.83      0.82       246
            misc.forsale       0.72      0.96      0.82       244
               rec.autos       0.85      0.88      0.87       244
         rec.motorcycles       0.86      0.95      0.90       238
      rec.sport.baseball       0.95      0.91      0.93       256
        rec.sport.hockey       0.91      0.92      0.92       247
               sci.crypt       0.98      0.96      0.97       260
         sci.electronics       0.86      0.

## 2. Implement Naive Bayes on your own from scratch for text classification

#### Note: As mentioned in the hint vedio I have ignored the probablities of those words in the testing data (while predicting the output) which are not present in our vocabulary

### Fit Function

In [12]:
def fit(x_train, y_train):
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    result = {}
    class_values = set(y_train)
    
    #Iterating over all possible class values
    for current_class in class_values:
        result[current_class] = {}
        #Getting total number of data points
        result["total_data"] = len(y_train)
        
        #Getting the data for the current class
        current_class_rows = (y_train == current_class)
        x_train_current = x_train[current_class_rows]
        y_train_current = y_train[current_class_rows]
        
        #Getting total number of data points in current class
        result[current_class]["total_count"] = len(y_train_current)
        
        #Finding the Number of Features(Words in Vocabulary)
        n = x_train.shape[1]
        
        #Storing the total Frequency of Each Word in the Current class
        for i in range(n):
            result[current_class][i] = x_train_current[:,i].sum()
                
    return result

In [13]:
def get_probability(dictionary, x, current_class):
    
    #Calculating the Prior Probabilty 
    output = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary["total_data"])
    
    #Finding the number of Features (Words in Vocabulary)
    num_of_features = len(dictionary[current_class].keys()) - 1
    
    #Iterating over the feature set (Words in Vocabulary) to calculate the probability of Each Feature (Word)
    for i in range(num_of_features):
        
        #Checking if the ith word in vocabulary is present in x or not
        if x[i] > 0:
            
            #Finding the count of feature (word) in the current class
            # Adding 1 for Laplace correction
            current_class_ith_word_count = dictionary[current_class][i] + 1
        
            #Getting the total count for current class
            count_current_class = dictionary[current_class]["total_count"] + len(dictionary[current_class].keys())
            
            #Calculating the probability of ith Feature (Word)
            current_xi_probablity = np.log(current_class_ith_word_count) - np.log(count_current_class)
            output = output + current_xi_probablity
            
    return output
    

In [14]:
def predict_for_one_point(dictionary, x):
    
    #Getting all possible class values
    classes_list = dictionary.keys()
    
    best_probality = -1000
    best_class = -1
    first_run = True
    
    #Iterating over all possible class values
    for current_class in classes_list:
        
        if (current_class == "total_data"):
            continue
        
        #Getting the probabilty of x belonging to the current class
        prob_current_class = get_probability(dictionary, x, current_class)
        
        #Finding the best class for x
        if (first_run or prob_current_class > best_probality):
            best_probality = prob_current_class
            best_class = current_class
            
        first_run = False
        
    return best_class

### Predict Function

In [15]:
def predict(dictionary, x_test):
    
    y_pred = []
    #Iterating over each data point in the testing dataset
    for x in x_test:
        x_class = predict_for_one_point(dictionary, x)
        y_pred.append(x_class)
        
    return y_pred

### Training

In [16]:
dictionary = fit(x_train_2d, y_train)

### Testing

In [17]:
y_pred_self = predict(dictionary, x_test_2d)

In [19]:
print("---------------------------------------------------------------------------")
print("Classification Report")
print(classification_report(y_test,y_pred_self))
print("---------------------------------------------------------------------------")
print("Confusion Matrix")
print(confusion_matrix(y_test,y_pred_self))
print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
Classification Report
                          precision    recall  f1-score   support

             alt.atheism       0.80      0.75      0.77       253
           comp.graphics       0.46      0.88      0.60       249
 comp.os.ms-windows.misc       0.76      0.70      0.73       233
comp.sys.ibm.pc.hardware       0.84      0.56      0.67       263
   comp.sys.mac.hardware       0.99      0.39      0.56       252
          comp.windows.x       0.70      0.95      0.81       246
            misc.forsale       0.93      0.59      0.73       244
               rec.autos       0.94      0.66      0.78       244
         rec.motorcycles       0.99      0.76      0.86       238
      rec.sport.baseball       0.99      0.73      0.84       256
        rec.sport.hockey       0.89      0.95      0.92       247
               sci.crypt       0.74      0.96      0.83       260
         sci.electronics       0.97      0.

## 3. Compare Results of your implementation of Naive Bayes with one in Sklearn

### Classification Report

In [20]:
print("---------------------------------------------------------------------------")
print("Classification Report of Naive Bayes already implemented in sklearn")
print(classification_report(y_test,y_pred))
print("---------------------------------------------------------------------------")
print("Classification Report of self Implemented Naive Bayes")
print(classification_report(y_test,y_pred_self))
print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
Classification Report of Naive Bayes already implemented in sklearn
                          precision    recall  f1-score   support

             alt.atheism       0.82      0.74      0.77       253
           comp.graphics       0.77      0.72      0.74       249
 comp.os.ms-windows.misc       0.83      0.06      0.12       233
comp.sys.ibm.pc.hardware       0.57      0.87      0.69       263
   comp.sys.mac.hardware       0.77      0.90      0.83       252
          comp.windows.x       0.81      0.83      0.82       246
            misc.forsale       0.72      0.96      0.82       244
               rec.autos       0.85      0.88      0.87       244
         rec.motorcycles       0.86      0.95      0.90       238
      rec.sport.baseball       0.95      0.91      0.93       256
        rec.sport.hockey       0.91      0.92      0.92       247
               sci.crypt       0.98      0.96      0.97       2

##### It can easily be observed that the overall performace of both self implemented Naive Bayes and Sklearn implemented Naive Bayes is more or less the same with one performing better for some classes (output) while other performing better for rest.

### Confusion Matrix

In [21]:
print("---------------------------------------------------------------------------")
print("Confusion Matrix of Naive Bayes already implemented in sklearn")
print(confusion_matrix(y_test,y_pred))
print("---------------------------------------------------------------------------")
print("Confusion Matrix of self Implemented Naive Bayes")
print(confusion_matrix(y_test,y_pred_self))
print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
Confusion Matrix of Naive Bayes already implemented in sklearn
[[186   0   0   0   0   0   0   3   2   1   1   1   0   1   0   0   1   1
    1  55]
 [  0 179   0  20  15   6  12   5   2   1   0   0   8   1   0   0   0   0
    0   0]
 [  0  27  15 116  11  41  17   1   0   0   0   1   4   0   0   0   0   0
    0   0]
 [  1   2   0 229  22   1   5   0   0   0   0   0   3   0   0   0   0   0
    0   0]
 [  0   3   1  13 226   0   6   0   0   0   0   0   3   0   0   0   0   0
    0   0]
 [  0  13   0  13   6 204   6   1   1   0   0   0   0   1   1   0   0   0
    0   0]
 [  0   1   0   3   0   0 234   2   2   0   0   0   2   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   9 215  13   0   4   0   2   0   0   0   1   0
    0   0]
 [  0   0   0   0   0   0   5   5 225   0   0   0   1   0   1   0   1   0
    0   0]
 [  0   0   0   0   0   0   3   2   3 234  13   0   0   0   0   0   1   0
    0   0]
 [  0   0  